In [6]:
import sys
sys.path.append( '../utilities' )
from data_engine import DataEngine
from backtesting import Backtesting
import pandas as pd
import ta
from IPython.display import clear_output
import ccxt
from datetime import datetime
from datetime import timedelta
import dateutil
import pandas_datareader.data as pdr
from stocktrends import Renko
import matplotlib.pyplot as plt
import matplotlib

In [7]:
# pairList = pairJson['hitbtc2018']
pair = 'ETH/USDT'
timeframe = '15m'
startDate = '2019-01-01T00:00:00'

dataEngine = DataEngine(session=ccxt.binance(), path_to_data='./database/')

df = dataEngine.get_historical_from_db(pair, timeframe, startDate)
df['date'] = df.index

print (df)

Successfully load 115180 candles for ETHUSDT
                        open     high      low    close      volume  \
timestamp                                                             
2019-01-01 00:00:00   131.45   131.54   131.01   131.31  3431.58885   
2019-01-01 00:15:00   131.30   131.44   131.25   131.29  1914.77814   
2019-01-01 00:30:00   131.28   131.81   131.16   131.78  3454.19200   
2019-01-01 00:45:00   131.78   132.22   131.61   131.82  3877.40277   
2019-01-01 01:00:00   131.83   131.88   131.26   131.65  2900.49848   
...                      ...      ...      ...      ...         ...   
2022-04-17 11:45:00  3044.88  3048.66  3043.54  3048.05  2500.62640   
2022-04-17 12:00:00  3048.05  3056.63  3044.05  3046.27  3599.95390   
2022-04-17 12:15:00  3046.28  3049.10  3042.80  3047.93   755.26890   
2022-04-17 12:30:00  3047.93  3051.19  3047.93  3050.78   578.01990   
2022-04-17 12:45:00  3053.28  3054.04  3051.03  3051.09   286.80850   

                               

In [8]:
# Function to convert ohlc data into renko bricks. Pass dataframe name and brick size
def df_to_renko(data, n):
    data.reset_index(inplace=True)
    data.columns = [i.lower() for i in data.columns]
    df = Renko(data)
    df.brick_size = n
    renko_df = df.get_ohlc_data()
    return renko_df

In [18]:
# -- Intern parameter --
dfTest = df['2020-01-01':]
#ndfTest = df[:]
startingBalance = 1000
makerFee = 0.0002
takerFee = 0.0007

# -- Trade Functions --
# -- Condition to BUY market --

def buyCondition(row):
    if row['uptrend'] == True:
        return True
    else:
        return False 

def sellCondition(row):
    if row['uptrend'] == False:
        return True
    else:
        return False


# -- Value initialisation --
usd = startingBalance
wallet = startingBalance
coin = 0
previousRow = dfTest.iloc[0]
timeSince = 0

# -- Definition of dfTrades, that will be the dataset to do your trades analyses --
dfTrades = None
dfTrades = pd.DataFrame(columns=['date', 'position', 'reason',
                                 'price', 'frais', 'fiat', 'coins', 'wallet', 'timeSince'])

# -- Iteration on all your price dataset (df) --
for index, row in dfTest.iterrows():
    clear_output(wait=True)
    print(index)
    startRenko = datetime.strptime(str(index), '%Y-%m-%d %H:%M:%S') - timedelta(minutes=15000)
    dfComputeRenko = df[startRenko:index]
    dfRenko = df_to_renko(dfComputeRenko, row['close']*0.02)

    print (dfRenko)

    timeSince += 1
    # -- Buy market order --
    if buyCondition(dfRenko.tail(1)) and usd > 0:
        # -- You can define here at what price you buy --
        buyPrice = row['close']

        fee = takerFee * usd
        usd = usd - fee
        coin = usd / buyPrice
        usd = 0
        wallet = coin * row['close']

        # -- Add the trade to dfTrades to analyse it later --
        myrow = {
            'date': index,
            'position': "Buy",
            'reason': 'Buy Market Order',
            'price': buyPrice,
            'frais': fee,
            'fiat': usd,
            'coins': coin,
            'wallet': wallet,
            'timeSince': timeSince
        }
        dfTrades = dfTrades.append(myrow, ignore_index=True)
        timeSince = 0

    # -- Sell Market Order --
    elif sellCondition(dfRenko.tail(1)):
        if coin > 0:
            # -- You can define here at what price you buy --
            sellPrice = row['close']
            usd = coin * sellPrice
            fee = takerFee * usd
            usd = usd - fee
            coin = 0
            wallet = usd

            # -- Add the trade to DfTrades to analyse it later --
            myrow = {
                'date': index,
                'position': "Sell",
                'reason': 'Sell Market Order',
                'price': sellPrice,
                'frais': fee,
                'fiat': usd,
                'coins': coin,
                'wallet': wallet,
                'timeSince': timeSince
            }
            dfTrades = dfTrades.append(myrow, ignore_index=True)
            timeSince = 0
    
    previousRow = row

print("Final wallet", wallet)
print(dfTrades)


2020-01-01 00:00:00
                  date      open      high       low     close  uptrend
0  2019-12-21 14:00:00  123.7536  126.3318  123.7536  126.3318     True
1  2019-12-22 07:45:00  126.3318  128.9100  126.3318  128.9100     True
2  2019-12-22 19:00:00  128.9100  131.4882  128.9100  131.4882     True
3  2019-12-23 01:00:00  131.4882  134.0664  131.4882  134.0664     True
4  2019-12-23 19:15:00  131.4882  131.4882  128.9100  128.9100    False
5  2019-12-25 01:00:00  128.9100  128.9100  126.3318  126.3318    False
6  2019-12-27 11:45:00  126.3318  126.3318  123.7536  123.7536    False
7  2019-12-28 17:45:00  126.3318  128.9100  126.3318  128.9100     True
8  2019-12-29 14:15:00  128.9100  131.4882  128.9100  131.4882     True
9  2019-12-29 19:45:00  131.4882  134.0664  131.4882  134.0664     True
10 2019-12-31 19:00:00  131.4882  131.4882  128.9100  128.9100    False
                  date      open      high     low   close  uptrend
10 2019-12-31 19:00:00  131.4882  131.4882  128.

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().